In [1]:
from corus import load_ne5
import os
import zipfile
import pickle
from collections import Counter
from typing import Optional
from dataclasses import dataclass

from sklearn.model_selection import train_test_split
from seqeval.metrics import classification_report
import evaluate

from transformers import TrainingArguments, Trainer, AutoTokenizer,  AutoModelForTokenClassification, AutoModelForMaskedLM, DataCollatorForLanguageModeling, DataCollatorForTokenClassification, pipeline
import datasets

import pandas as pd
import numpy as np

In [2]:
model_path = 'cointegrated/rubert-tiny2'
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [3]:
reports = {}

# 1. Train NER Model

## 1.1 Load Collections5 Dataset

In [4]:
path = 'collection5.zip'
directory = 'Collection5'

if not os.path.exists(path):
    !wget http://www.labinform.ru/pub/named_entities/collection5.zip
    with zipfile.ZipFile(path, 'r') as f:
        f.extractall()
        print("Unzipped.")
else:
    print("Already loaded.")

Already loaded.


In [5]:
records = []

for recode in load_ne5(directory):
    records.append(
        {
            'text': recode.text,
            'id': recode.id,
            'spans': recode.spans
        }
    )

In [6]:
def labeling(text: str, spans: list):
    tokens = tokenizer.tokenize(text)
    labels = ['O'] * len(tokens)
    for span in spans:
        start, end, label = span.start, span.stop, span.type
        span_tokens = tokenizer.tokenize(text[start:end])
        for j in range(len(tokens)):
            if tokens[j:j+len(span_tokens)] == span_tokens:
                labels[j] = f'B-{label}'
                labels[j+1:j+len(span_tokens)
                       ] = [f'I-{label}'] * (len(span_tokens) - 1)
    return {"text": text, "tokens": tokens, "ner_tags": labels}

In [7]:
raw_dataset = pd.DataFrame(list(map(lambda x: labeling(x['text'], x['spans']), records)))

Token indices sequence length is longer than the specified maximum sequence length for this model (2306 > 2048). Running this sequence through the model will result in indexing errors


In [8]:
raw_dataset

,text,tokens,ner_tags
0,В Санкт-Петербурге создают госкорпорацию промы...,"[В, Санкт, -, Петербурге, создают, госкорпора,...","[O, B-LOC, I-LOC, I-LOC, O, O, O, O, O, O, O, ..."
1,Генпрокуратура Египта распорядилась закрыть оп...,"[Генпроку, ##ратура, Египта, распоряди, ##лась...","[B-ORG, I-ORG, B-GEOPOLIT, O, O, O, O, O, O, O..."
2,Глава Скотланд-Ярда ушел в отставку из за скан...,"[Глава, Ско, ##тла, ##нд, -, Яр, ##да, ушел, в...","[O, B-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, ..."
3,Д.Медведев уволил главу Ярославской области\r\...,"[Д, ., Медведев, уволил, главу, Ярославской, о...","[B-PER, I-PER, I-PER, O, O, B-LOC, I-LOC, O, B..."
4,Президент Мексики предложил переименовать стра...,"[Президент, Мексики, предложил, переименов, ##...","[O, B-GEOPOLIT, O, O, O, O, O, B-GEOPOLIT, O, ..."
...,...,...,...
995,Новым главой МИД Египта стал Мухаммед Камель А...,"[Новым, главой, МИД, Египта, стал, Мухаммед, К...","[O, O, B-ORG, B-GEOPOLIT, O, B-PER, I-PER, I-P..."
996,"""Ювентус"" обыграл ""Милан"" в Кубке Италии\r\n""Ю...","["", Ювентус, "", обыграл, "", Милан, "", в, Кубке...","[O, B-ORG, O, O, O, B-ORG, O, O, O, B-GEOPOLIT..."
997,\r\nЛавров сохранил пост главы комиссии по вза...,"[Лавров, сохранил, пост, главы, комиссии, по, ...","[B-PER, O, O, O, O, O, O, O, B-ORG, O, O, O, B..."
998,Чавес назначил нового вице-президента\r\nПрези...,"[Ч, ##аве, ##с, назначил, нового, вице, -, пре...","[B-PER, I-PER, I-PER, O, O, O, O, O, O, B-GEOP..."


Загрузили датасет

## 1.2 Train/Test split

Разделяем данные на train/test

In [9]:
RANDOM_SEED = 42
TEST_SAMPLE_SIZE = 0.2

raw_train, raw_test = train_test_split(raw_dataset, test_size=TEST_SAMPLE_SIZE, random_state=RANDOM_SEED)

In [10]:
assert set(raw_train['ner_tags'].sum()) == set(raw_test['ner_tags'].sum()), 'Some of tags not in both train/test'

train_tags_cnt = Counter(raw_train['ner_tags'].sum())
test_tags_cnt = Counter(raw_dataset['ner_tags'].sum())

for tag in train_tags_cnt.keys():
    print(f'Train freq: {train_tags_cnt[tag] / sum(train_tags_cnt.values()):.4f}, Test freq: {test_tags_cnt[tag] / sum(test_tags_cnt.values()):.4f} Tag: {tag}: ')


Train freq: 0.7777, Test freq: 0.7774 Tag: O: 
Train freq: 0.0049, Test freq: 0.0049 Tag: B-MEDIA: 
Train freq: 0.0055, Test freq: 0.0054 Tag: I-MEDIA: 
Train freq: 0.0107, Test freq: 0.0103 Tag: B-LOC: 
Train freq: 0.0376, Test freq: 0.0378 Tag: B-PER: 
Train freq: 0.0758, Test freq: 0.0767 Tag: I-PER: 
Train freq: 0.0133, Test freq: 0.0134 Tag: B-GEOPOLIT: 
Train freq: 0.0024, Test freq: 0.0024 Tag: I-GEOPOLIT: 
Train freq: 0.0231, Test freq: 0.0232 Tag: B-ORG: 
Train freq: 0.0368, Test freq: 0.0370 Tag: I-ORG: 
Train freq: 0.0122, Test freq: 0.0116 Tag: I-LOC: 


In [11]:
tags = sorted(train_tags_cnt.keys())
id2tag = {i: tag for i, tag in enumerate(tags)}
tag2id = {tag: i for i, tag in enumerate(tags)}

In [12]:
features = datasets.Features({
    "tokens": datasets.Sequence(feature=datasets.Value("string")),
    "ner_tags": datasets.Sequence(feature=datasets.ClassLabel(names=tags))
})

train = datasets.Dataset.from_dict({'tokens': raw_train['tokens'], 'ner_tags': raw_train['ner_tags']}, features=features)
test = datasets.Dataset.from_dict({'tokens': raw_test['tokens'], 'ner_tags': raw_test['ner_tags']}, features=features)

Разбили данные на train/test 

Удастоверились, что частотность меток в train/test примерно одинаковое  


## 1.3 Train model for NER task

In [ ]:
simple_model = AutoModelForTokenClassification.from_pretrained(
    model_path,
    num_labels=len(tags),
    id2label=id2tag,
    label2id=tag2id,
)

In [14]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True,
        padding='max_length',
        max_length=128
    )
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_train = train.map(tokenize_and_align_labels, batched=True)
tokenized_test = test.map(tokenize_and_align_labels, batched=True)
data_collator = DataCollatorForTokenClassification(tokenizer)

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [15]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    save_strategy="no"
)

simple_trainer = Trainer(
    model=simple_model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    data_collator=data_collator,
    processing_class=tokenizer
)

In [16]:
#init metrics
def report(trainer: Trainer):
    predictions, labels, loss = trainer.predict(tokenized_test)
    predictions = np.argmax(predictions, axis=2)

    true_labels = [[id2tag[l] for l in label if l != -100] for label in labels]
    pred_labels = [[id2tag[p] for p, l in zip(pred, label) if l != -100] 
                for pred, label in zip(predictions, labels)]

    return classification_report(true_labels, pred_labels)

rprt = report(simple_trainer)
reports['simple_model_before_train'] = rprt

print(rprt)


              precision    recall  f1-score   support

    GEOPOLIT       0.03      0.45      0.06       388
         LOC       0.01      0.10      0.01       235
       MEDIA       0.01      0.06      0.02       153
         ORG       0.00      0.01      0.00       540
         PER       0.01      0.02      0.01       934

   micro avg       0.02      0.10      0.03      2250
   macro avg       0.01      0.13      0.02      2250
weighted avg       0.01      0.10      0.02      2250



In [17]:
_ = simple_trainer.train()

Epoch,Training Loss,Validation Loss,Model Preparation Time
1,No log,1.016602,0.000700
2,No log,0.694065,0.000700
3,No log,0.548105,0.000700
4,No log,0.471383,0.000700
5,No log,0.423856,0.000700
6,No log,0.385486,0.000700
7,No log,0.362924,0.000700
8,No log,0.348801,0.000700
9,No log,0.340781,0.000700
10,0.600100,0.337425,0.000700


In [18]:
rprt = report(simple_trainer)
reports['simple_model_after_train'] = rprt

print(rprt)

              precision    recall  f1-score   support

    GEOPOLIT       0.80      0.84      0.82       388
         LOC       0.53      0.50      0.52       235
       MEDIA       0.48      0.27      0.34       153
         ORG       0.39      0.48      0.43       540
         PER       0.51      0.65      0.57       934

   micro avg       0.52      0.60      0.56      2250
   macro avg       0.54      0.55      0.54      2250
weighted avg       0.53      0.60      0.56      2250



Сейчас и в дальнейшем смотрим на ___F1 weighted___ 

Вимим, что модель обучилась, причем относительно нормально 

Далее будем пытаться улучшить результат

# 2. Improvement

## 2.1 MLM->NER

Загружаем модель в режиме MLM

In [75]:
mlm_model = AutoModelForMaskedLM.from_pretrained(model_path)

In [76]:
mlm_train_text = datasets.Dataset.from_dict({"text": raw_train['text'].tolist()})

def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        max_length=512,
        padding="max_length",
        return_special_tokens_mask=True
    )

mlm_train = mlm_train_text.map(tokenize_function, batched=True)

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

In [77]:
MLM_PROBABILITY = 0.15

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm_probability=MLM_PROBABILITY,
    seed=42
)

In [91]:
def add_mask(text: str) -> tuple[str, str]:
    words = text.split()
    masked_id = np.random.randint(0, len(words) - 1)
    masked_word = words[masked_id]
    words[masked_id] = "[MASK]"
    return " ".join(words), masked_word

mlm_test = [add_mask(text) for text in raw_test['text'].tolist()]

def calculate_top_recalls(mlm_model, ns=[1, 5]):
    correct = [0]*len(ns)
    total = 0
    mask_filler = pipeline("fill-mask", model=mlm_model, tokenizer=tokenizer)
    for text, masked_word in mlm_test:
        predictions = mask_filler(text, top_k=max(ns))
        predicted_words = [predictions[i]['token_str'] for i in range(max(ns))]
        for i, n in enumerate(ns):
            if masked_word in predicted_words[:n]:
                correct[i] += 1 
        total += 1
    return [c / total for c in correct]



In [ ]:
acc, rec = calculate_top_recalls(mlm_model, ns=[1, 5])

print(f'Initial MLM accuracy: {acc}, Recall@5: {rec}')

Device set to use cuda:0


Initial MLM accuracy: 0.285, Recall@5: 0.435


In [95]:
mlm_training_args = TrainingArguments(
    output_dir="./mlm_results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    learning_rate=2e-5,
)

mlm_trainer = Trainer(
    model=mlm_model,
    args=mlm_training_args,
    train_dataset=mlm_train,
    data_collator=data_collator,
)

_ = mlm_trainer.train()

Step,Training Loss


In [96]:
mlm_model.save_pretrained("./mlm-rubert-tiny2")

In [97]:
acc, rec = calculate_top_recalls(mlm_model, ns=[1, 5])

print(f'Trained MLM accuracy: {acc}, Recall@5: {rec}')

Device set to use cuda:0


Trained MLM accuracy: 0.315, Recall@5: 0.47


Видим, что уже изночально модель не ужасно решает MLM задачу


Также по разнице в метриках до\после обучения можно понять, что модель немного обучилась

Дальшейшее решение задачи mlm затруднительно, так как количество данных низкое для данной задачи 

In [ ]:
form_mlm_model = AutoModelForTokenClassification.from_pretrained(
    './mlm-rubert-tiny2',
    num_labels=len(tags),
    id2label=id2tag,
    label2id=tag2id,
)

In [31]:
training_args = TrainingArguments(
    output_dir="./mlm_ner_res",
    eval_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    save_strategy="no"
)

from_mlm_trainer = Trainer(
    model=form_mlm_model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    data_collator=data_collator,
    processing_class=tokenizer
)


In [32]:
rprt = report(from_mlm_trainer)
reports['form_mlm_before_train'] = rprt

print(rprt)

              precision    recall  f1-score   support

    GEOPOLIT       0.03      0.36      0.05       388
         LOC       0.01      0.08      0.02       235
       MEDIA       0.00      0.01      0.00       153
         ORG       0.00      0.00      0.00       540
         PER       0.01      0.02      0.01       934

   micro avg       0.01      0.08      0.02      2250
   macro avg       0.01      0.10      0.02      2250
weighted avg       0.01      0.08      0.01      2250



In [33]:
_ = from_mlm_trainer.train()

Epoch,Training Loss,Validation Loss,Model Preparation Time
1,No log,0.976450,0.001000
2,No log,0.645057,0.001000
3,No log,0.516093,0.001000
4,No log,0.443613,0.001000
5,No log,0.397509,0.001000
6,No log,0.363229,0.001000
7,No log,0.343079,0.001000
8,No log,0.330035,0.001000
9,No log,0.323961,0.001000
10,0.577700,0.320627,0.001000


In [34]:
rprt = report(from_mlm_trainer)
reports['form_mlm_after_train'] = rprt

print(rprt)

              precision    recall  f1-score   support

    GEOPOLIT       0.86      0.82      0.84       388
         LOC       0.59      0.47      0.53       235
       MEDIA       0.69      0.42      0.52       153
         ORG       0.40      0.54      0.46       540
         PER       0.51      0.66      0.57       934

   micro avg       0.54      0.62      0.58      2250
   macro avg       0.61      0.58      0.58      2250
weighted avg       0.56      0.62      0.58      2250



Видно, что дообучение в mlm дало прирост в метриках, но незначительный (так как метрики MLM задачи также не очень большие)

## 2.2 Argumentation

In [ ]:
!cd deeppavlov && bash run.sh

In [35]:
ag_tokens, ag_tags = pickle.load(open('deeppavlov/ner.pkl', 'rb'))

In [36]:
" ".join(ag_tokens[0]), " ".join(ag_tags[0])

('Названы регионы России с \xa0 самой высокой смертностью от \xa0 рака',
 'O O S-LOC O O O O O O O O')

In [41]:
ag_tags_cnt = Counter()
for t in ag_tags:
    ag_tags_cnt.update(t)
_total = sum(ag_tags_cnt.values())
for tag, count in ag_tags_cnt.items():
    print(f'{count / _total:.4f} :{tag}')

0.8840 :O
0.0449 :S-LOC
0.0179 :S-ORG
0.0273 :S-PER
0.0027 :B-LOC
0.0028 :E-LOC
0.0033 :B-ORG
0.0005 :I-ORG
0.0033 :E-ORG
0.0064 :B-PER
0.0066 :E-PER
0.0001 :I-LOC
0.0002 :I-PER


Видим, что больше всего бы дадим данныз для локаций (`LOC`) и организаций (`ORG`) 

In [42]:
deeppavlov2collections_ner_tags = {
    'O': 'O',
    'S-LOC': 'LOC',
    'S-PER': 'PER',
    'S-ORG': 'ORG',
    'E-PER': 'PER',
    'B-PER': 'PER',
    'B-ORG': 'ORG',
    'E-ORG': 'ORG',
    'E-LOC': 'LOC',
    'B-LOC': 'LOC',
    'I-ORG': 'ORG',
    'I-PER': 'PER',
    'I-LOC': 'LOC',
}

In [43]:
@dataclass(slots=True)
class Span:
    start: int
    stop: int
    type: str

@dataclass(frozen=True, slots=True)
class Row:
    text: str
    spans: list[Span]

def _resolve_spans(spans: list[Span]) -> list[Span]:
    i = 0

    while i < len(spans) - 1:
        if spans[i].stop + 1 == spans[i+1].start and spans[i].type == spans[i+1].type:
            spans[i].stop = spans[i+1].stop
            spans.pop(i+1)
            _resolve_spans(spans)
        i += 1
    return spans
    

def make_dataset_row(tokens: list[str], tags: list[str]) -> Optional[Row]:
    spans = []
    curr_len = 0
    for token, tag in zip(tokens, tags):
        if tag == 'O':
            curr_len += len(token) + 1
        else:
            spans.append(Span(start=curr_len, stop=curr_len + len(token), type=deeppavlov2collections_ner_tags[tag]))
            curr_len += len(token) + 1
    if len(spans) == 0:
        return None
    return Row(text=" ".join(tokens), spans=_resolve_spans(spans))

        



In [44]:
spans = [
    Span(start=0, stop=1, type='PER'),
    Span(start=2, stop=3, type='PER'),

    Span(start=24, stop=25, type='LOC'),

    Span(start=26, stop=27, type='PER'),
 
    Span(start=32, stop=33, type='PER'),
    Span(start=34, stop=35, type='PER'),
]
assert _resolve_spans(spans) == [Span(start=0, stop=3, type='PER'), Span(start=24, stop=25, type='LOC'), Span(start=26, stop=27, type='PER'), Span(start=32, stop=35, type='PER')] 

In [46]:
argumentation = []

for tkn, tg in zip(ag_tokens, ag_tags):
    row = make_dataset_row(tkn, tg)
    if row is not None:
        argumentation.append(row)


In [47]:
argumentation_raw_dataset = pd.DataFrame(list(map(lambda x: labeling(x.text, x.spans), argumentation)))

In [48]:
argumentation_raw_dataset

,text,tokens,ner_tags
0,Названы регионы России с самой высокой смерт...,"[Названы, регионы, России, с, самой, высокой, ...","[O, O, B-LOC, O, O, O, O, O, O, O]"
1,Австрия не представила доказательств вины ро...,"[Австрия, не, представила, доказательств, вины...","[B-LOC, O, O, O, O, O, O, O]"
2,В США раскрыли сумму расходов на расследован...,"[В, США, раскрыли, сумму, расходов, на, рассле...","[O, B-LOC, O, O, O, O, O, O, O, O, O]"
3,Хакеры рассказали о планах Великобритании за...,"[Хак, ##еры, рассказали, о, планах, Великобрит...","[O, O, O, O, O, B-LOC, O, O, B-LOC]"
4,Архиепископ канонической УПЦ отказался прийти ...,"[Архие, ##пископ, канони, ##ческой, УПЦ, отказ...","[O, O, O, O, B-ORG, O, O, O, O, O, O, O, O, O,..."
...,...,...,...
6200,МИД Сирии определило предназначение С - 300,"[МИД, Сирии, определило, предназначение, С, -,...","[B-ORG, B-LOC, O, O, O, O, O]"
6201,Памятный знак белому генералу на родине Лени...,"[Памят, ##ный, знак, белом, ##у, генералу, на,...","[O, O, O, O, O, O, O, O, B-PER, O, O, O]"
6202,В Польше заявили о конфликте идентичностей с...,"[В, Польше, заявили, о, конфликте, иденти, ##ч...","[O, B-LOC, O, O, O, O, O, O, O, B-LOC]"
6203,Возможность морской блокады России назвали фан...,"[Возможность, морской, блокады, России, назвал...","[O, O, O, B-LOC, O, O, O]"


In [49]:
augmented_raw_train = pd.concat([argumentation_raw_dataset, raw_train], axis=0)
augmented_raw_train

,text,tokens,ner_tags
0,Названы регионы России с самой высокой смерт...,"[Названы, регионы, России, с, самой, высокой, ...","[O, O, B-LOC, O, O, O, O, O, O, O]"
1,Австрия не представила доказательств вины ро...,"[Австрия, не, представила, доказательств, вины...","[B-LOC, O, O, O, O, O, O, O]"
2,В США раскрыли сумму расходов на расследован...,"[В, США, раскрыли, сумму, расходов, на, рассле...","[O, B-LOC, O, O, O, O, O, O, O, O, O]"
3,Хакеры рассказали о планах Великобритании за...,"[Хак, ##еры, рассказали, о, планах, Великобрит...","[O, O, O, O, O, B-LOC, O, O, B-LOC]"
4,Архиепископ канонической УПЦ отказался прийти ...,"[Архие, ##пископ, канони, ##ческой, УПЦ, отказ...","[O, O, O, O, B-ORG, O, O, O, O, O, O, O, O, O,..."
...,...,...,...
106,Экс-глава московского ОМОНа возглавил столичны...,"[Экс, -, глава, московского, ОМОН, ##а, возгла...","[O, O, O, O, B-ORG, I-ORG, O, O, B-ORG, I-ORG,..."
270,Обама назначил нового главу своей администраци...,"[Обама, назначил, нового, главу, своей, админи...","[B-PER, O, O, O, O, O, O, B-GEOPOLIT, B-PER, I..."
860,Центр ядерной медицины и мордовский могильник:...,"[Центр, ядерной, медицины, и, морд, ##овский, ...","[B-LOC, O, O, O, O, O, O, O, O, O, B-MEDIA, B-..."
435,Д.Медведев 18-19 января посетит Иорданское Хаш...,"[Д, ., Медведев, 18, -, 19, января, посетит, И...","[B-PER, I-PER, I-PER, O, O, O, O, O, B-GEOPOLI..."


In [50]:
features = datasets.Features({
    "tokens": datasets.Sequence(feature=datasets.Value("string")),
    "ner_tags": datasets.Sequence(feature=datasets.ClassLabel(names=tags))
})

augmented_train = datasets.Dataset.from_dict({'tokens': augmented_raw_train['tokens'], 'ner_tags': augmented_raw_train['ner_tags']}, features=features)

tokenized_augmented_train = augmented_train.map(tokenize_and_align_labels, batched=True)


Map:   0%|          | 0/7005 [00:00<?, ? examples/s]

In [ ]:
augmented_model = AutoModelForTokenClassification.from_pretrained(
    model_path,
    num_labels=len(tags),
    id2label=id2tag,
    label2id=tag2id,
)


In [57]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    save_strategy="no"
)

augmented_trainer = Trainer(
    model=augmented_model,
    args=training_args,
    train_dataset=tokenized_augmented_train,
    eval_dataset=tokenized_test,
    data_collator=data_collator,
    processing_class=tokenizer
)

_ = augmented_trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,0.425763
2,0.651800,0.291468
3,0.271500,0.237805
4,0.191600,0.214048
5,0.150600,0.196375
6,0.132400,0.188109
7,0.114500,0.184297
8,0.104300,0.180256
9,0.104300,0.178005
10,0.096700,0.177598


In [58]:
rprt = report(augmented_trainer)
reports['augmented_after_train'] = rprt

print(rprt)

              precision    recall  f1-score   support

    GEOPOLIT       0.89      0.87      0.88       388
         LOC       0.66      0.78      0.72       235
       MEDIA       0.79      0.79      0.79       153
         ORG       0.57      0.71      0.63       540
         PER       0.66      0.69      0.67       934

   micro avg       0.68      0.74      0.71      2250
   macro avg       0.71      0.77      0.74      2250
weighted avg       0.69      0.74      0.71      2250



Получаем сильный прирост в метриках. Основной гипотезой считаю, что мы дали значительно больше данных и по этому общее распределение в датасете о языке (гениральной совокупности в нашем случае) стало качественее 

# 3. Comparison

In [63]:
for name, rprt in reports.items():
    print(f"______________{name.replace('_', ' ').title()}______________", )
    print(rprt)

______________Simple Model Before Train______________
              precision    recall  f1-score   support

    GEOPOLIT       0.03      0.45      0.06       388
         LOC       0.01      0.10      0.01       235
       MEDIA       0.01      0.06      0.02       153
         ORG       0.00      0.01      0.00       540
         PER       0.01      0.02      0.01       934

   micro avg       0.02      0.10      0.03      2250
   macro avg       0.01      0.13      0.02      2250
weighted avg       0.01      0.10      0.02      2250

______________Simple Model After Train______________
              precision    recall  f1-score   support

    GEOPOLIT       0.80      0.84      0.82       388
         LOC       0.53      0.50      0.52       235
       MEDIA       0.48      0.27      0.34       153
         ORG       0.39      0.48      0.43       540
         PER       0.51      0.65      0.57       934

   micro avg       0.52      0.60      0.56      2250
   macro avg       0.54 

По итогу имеем
1. В ner задаче без всего имеем ___(baseline)___:
    - _F1 weighted_=0.56  
    - лучше все предсказания на классе __GEOPOLIT__ (_f1-score_ = 0.82)
    - на остальных картина не очень
2. в ner задаче с моделью, предобученной в mlm имеем ___(from mlm)___
    - незначительный прирост _F1 weighted_=0.58 ($\Delta_{baseline} = 2\% $)
    - локально на классе __MEDIA__ с худшими метриками в ___baseline___ стали сильно лучше метрики 
    - слабый прирост объясняется сложностями в MLM задаче на малых данных
3. в ner задаче с аугментацией имеем ___(augmented)___
    - нучшие метрики _F1 weighted_=0.71 ($\Delta_{baseline} = 15\% $)
    - значительно лучше предсказываем на всех классах
    - сильный прирост объясняется хначительным увеличением тренировочных данных (лучшее представление о языке) 